# Chapter 48: Security in CI/CD

While velocity drives modern DevOps practices, security must evolve from a final gate to an integrated, continuous process. Traditional security models—where code undergoes lengthy audits before production—collapse under the speed of CI/CD pipelines. This chapter establishes **DevSecOps** principles: embedding security controls throughout the software supply chain, from developer IDE to production runtime. We examine how to secure the pipeline itself (supply chain security), verify artifact integrity (signing and SBOMs), manage credentials without exposure, enforce policy-as-code, and respond to vulnerabilities in an automated, continuous manner. These practices align with industry frameworks including SLSA (Supply Chain Levels for Software Artifacts), NIST SSDF (Secure Software Development Framework), and CIS Benchmarks.

## 48.1 Supply Chain Security

Software supply chain security addresses the integrity of everything that touches your code from commit to deployment. Modern applications depend on hundreds of dependencies, build tools, and infrastructure components—each a potential attack vector.

### The Supply Chain Threat Model

**Upstream Risks:**
- **Dependency Confusion**: Attackers publish malicious packages with names similar to internal libraries to public repositories
- **Typosquatting**: Malicious packages exploiting typing errors (e.g., `reqeusts` instead of `requests`)
- **Compromised Maintainers**: Legitimate package maintainers with compromised credentials
- **Build System Compromise**: Malicious code injection during compilation (SolarWinds-style attacks)

**Pipeline Risks:**
- **Poisoned Pipeline Execution**: Malicious PRs executing code in CI environments with elevated privileges
- **Credential Leakage**: Secrets embedded in logs, artifacts, or environment variables
- **Insecure Defaults**: Using `latest` tags or unsigned base images
- **Privilege Escalation**: Containers running as root with host access

**Distribution Risks:**
- **Man-in-the-Middle**: Image tampering during transit to registry
- **Registry Compromise**: Unauthorized image pushes or metadata modifications
- **Replay Attacks**: Deployment of old vulnerable versions

### Defense in Depth Strategy

Implement **Zero Trust** for your pipeline: never trust, always verify.

```yaml
# Example: Secure pipeline architecture
Pipeline Security Layers:
  1. Source Integrity (Signed commits, branch protection)
  2. Dependency Scanning (SCA - Software Composition Analysis)
  3. Build Isolation (Ephemeral, minimal privilege runners)
  4. Artifact Signing (Cryptographic verification)
  5. Image Hardening (Distroless, non-root, read-only)
  6. Runtime Protection (Policy enforcement, monitoring)
```

### Securing the Source

**Branch Protection Rules** (GitHub/GitLab):
```yaml
# GitHub branch protection policy (procedural, configure in UI or Terraform)
main_branch_protection:
  required_pull_request_reviews:
    required_approving_review_count: 2
    dismiss_stale_reviews: true
    require_code_owner_reviews: true
  required_status_checks:
    strict: true
    contexts:
      - "ci/tests"
      - "security/scan"
  required_signatures: true  # Commit signing required
  enforce_admins: true
  restrictions:
    users: []  # Only allow teams
    teams: ["platform-engineering"]
```

**Commit Signing with GPG/SSH**:
```bash
# Configure Git to sign all commits
git config --global commit.gpgsign true
git config --global user.signingkey <KEY_ID>

# Verify signature on pull
git verify-commit HEAD

# In CI: Reject unsigned commits
if ! git verify-commit HEAD --verbose 2>/dev/null; then
  echo "ERROR: Unsigned commit detected"
  exit 1
fi
```

### Build Environment Isolation

**Ephemeral Build Pods** (Kubernetes-native CI):
```yaml
apiVersion: v1
kind: Pod
metadata:
  name: secure-builder
spec:
  containers:
  - name: build
    image: gcr.io/distroless/nodejs20-debian12  # Minimal attack surface
    securityContext:
      runAsNonRoot: true
      runAsUser: 65532
      readOnlyRootFilesystem: true
      allowPrivilegeEscalation: false
      capabilities:
        drop:
        - ALL
      seccompProfile:
        type: RuntimeDefault
    resources:
      limits:
        cpu: "2000m"
        memory: "4Gi"
      requests:
        cpu: "1000m"
        memory: "2Gi"
  restartPolicy: Never
  automountServiceAccountToken: false  # Prevent Kubernetes API access
  securityContext:
    fsGroup: 65532
```

**Container Sandboxing**:
Use gVisor or Kata Containers for untrusted build steps:
```yaml
# RuntimeClass for gVisor (sandboxed containers)
apiVersion: node.k8s.io/v1
kind: RuntimeClass
metadata:
  name: gvisor
handler: runsc
---
# Usage in Pod spec
spec:
  runtimeClassName: gvisor  # Isolates build from host kernel
```

## 48.2 SBOM (Software Bill of Materials)

An SBOM is a formal, machine-readable inventory of software components and dependencies. Think of it as a nutritional label for your software, listing every ingredient (library, package, operating system component) and its origin.

### SBOM Standards

**SPDX (Software Package Data Exchange)**:
- Linux Foundation standard
- Supports complex relationships (contains, depends on, describes)
- File format: Tag-value, JSON, RDF, YAML
- Use case: Legal compliance, license tracking

**CycloneDX**:
- OWASP standard
- Optimized for security use cases
- Native vulnerability correlation
- File format: JSON, XML, Protobuf
- Use case: Security auditing, vulnerability management

**SWID (Software Identification Tags)**:
- ISO/IEC 19770-2 standard
- Focus on software asset management
- Use case: Enterprise inventory, procurement

### Generating SBOMs in CI

**Syft (Anchore)** - Universal SBOM generator:
```yaml
# GitHub Actions example
- name: Generate SBOM
  uses: anchore/sbom-action@v0
  with:
    image: "myapp:${{ github.sha }}"
    format: spdx-json
    output-file: sbom.spdx.json
    
- name: Generate CycloneDX
  uses: anchore/sbom-action@v0
  with:
    image: "myapp:${{ github.sha }}"
    format: cyclonedx-json
    output-file: sbom.cdx.json
```

**Trivy** - Integrated scanning and SBOM:
```bash
# Generate SBOM during build
trivy image --format cyclonedx --output sbom.json myapp:latest

# Include in artifact storage
aws s3 cp sbom.json s3://sbom-bucket/myapp/${GITHUB_SHA}/sbom.json

# Verify in deployment pipeline
trivy sbom --severity HIGH,CRITICAL sbom.json
```

**Docker Buildx** (Built-in SBOM):
```bash
# Build with SBOM attestation
docker buildx build \
  --sbom=true \
  --attest type=sbom \
  --output type=image,name=myapp:latest,push=true \
  .
```

### SBOM Content Structure

**CycloneDX Example** (simplified):
```json
{
  "bomFormat": "CycloneDX",
  "specVersion": "1.5",
  "serialNumber": "urn:uuid:3e671687-395b-41f5-a30f-a5d638c93d26",
  "version": 1,
  "components": [
    {
      "type": "library",
      "name": "log4j-core",
      "version": "2.14.1",
      "purl": "pkg:maven/org.apache.logging.log4j/log4j-core@2.14.1",
      "licenses": [{"license": {"id": "Apache-2.0"}}],
      "properties": [
        {"name": "syft:package:foundBy", "value": "java-cataloger"}
      ]
    },
    {
      "type": "container",
      "name": "myapp",
      "version": "1.2.3",
      "purl": "pkg:oci/myapp@sha256:abc123...",
      "properties": [
        {"name": "arch", "value": "amd64"},
        {"name": "os", "value": "debian"},
        {"name": "os_version", "value": "12"}
      ]
    }
  ],
  "dependencies": [
    {
      "ref": "pkg:maven/org.apache.logging.log4j/log4j-core@2.14.1",
      "dependsOn": [
        "pkg:maven/org.apache.logging.log4j/log4j-api@2.14.1"
      ]
    }
  ]
}
```

### Storing and Distributing SBOMs

**OCI Registry Storage** (Attach SBOM to image):
```bash
# Oras CLI for attaching artifacts
oras attach \
  --artifact-type application/vnd.cyclonedx+json \
  myregistry/myapp:1.0 \
  ./sbom.json

# Discover attached SBOMs
oras discover myregistry/myapp:1.0
```

**Retrieval in Production**:
```bash
# Pull SBOM before deployment approval
cosign download sbom \
  --output-file sbom.json \
  myregistry/myapp:1.0

# Policy check: Block if SBOM missing
if [ ! -f sbom.json ]; then
  echo "SBOM required for deployment"
  exit 1
fi
```

## 48.3 Image Signing and Verification

Cryptographic signing ensures that the container image deployed to production is exactly what was built and approved in CI, untampered during transit or storage.

### Sigstore and Cosign

**Sigstore** is a Linux Foundation project providing software signing infrastructure without keys. **Cosign** is the container signing tool.

**Keyless Signing** (Recommended):
Uses ephemeral keys and identity federation (OIDC) rather than long-lived keys.

```bash
# Sign image using OIDC identity (GitHub Actions, etc.)
cosign sign \
  --yes \
  --identity-token $(cat $GITHUB_TOKEN_PATH) \
  myregistry/myapp@${DIGEST}

# Verify signature
cosign verify \
  --certificate-identity-regexp "https://github.com/myorg/myapp" \
  --certificate-oidc-issuer-regexp "https://token.actions.githubusercontent.com" \
  myregistry/myapp:latest
```

**Traditional Key-based Signing**:
```bash
# Generate key pair (store securely, e.g., KMS)
cosign generate-key-pair

# Sign with private key
cosign sign --key cosign.key myregistry/myapp:1.0

# Verify with public key
cosign verify --key cosign.pub myregistry/myapp:1.0
```

### Kubernetes Admission Control

**Policy Controller** (Sigstore) or **Kyverno** enforce signature verification before Pod creation.

**Kyverno Policy**:
```yaml
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: verify-image-signature
spec:
  validationFailureAction: Enforce
  rules:
  - name: check-image-signature
    match:
      resources:
        kinds:
        - Pod
    verifyImages:
    - imageReferences:
      - "myregistry.io/myapp*"
      attestors:
      - entries:
        - keys:
            publicKeys: |
              -----BEGIN PUBLIC KEY-----
              MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE8n...
              -----END PUBLIC KEY-----
      mutateDigest: true  # Replace tag with digest
      verifyDigest: true  # Ensure digest matches
```

**Gatekeeper (OPA) Policy**:
```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sVerifiedImage
metadata:
  name: verify-image-signature
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Pod"]
  parameters:
    images:
      - myregistry.io/myapp
    requiredSignature:
      - keySecret: cosign-public-key
        signatureAlgorithm: cosign
```

### In-Toto Attestations

For complex supply chains, use **in-toto** to track each step (build, test, sign) with signed metadata.

```bash
# Create attestation for build step
in-toto-record start \
  --step-name build \
  --key private.pem \
  --materials Dockerfile,src/ \
  --products dist/app.jar

# Complete step
in-toto-record stop \
  --step-name build \
  --key private.pem \
  --products dist/app.jar
```

## 48.4 Secrets Management

While Chapter 49 covers secrets management in depth, CI/CD pipelines require specific secret handling patterns to prevent exposure in logs, artifacts, and environment variables.

### Pipeline Secret Injection

**Avoid**: Hardcoded secrets, `.env` files in repos, base64 encoding in YAML.

**GitHub Actions** (Encrypted Secrets):
```yaml
jobs:
  build:
    steps:
    - name: Build with secrets
      env:
        DATABASE_PASSWORD: ${{ secrets.DB_PASSWORD }}  # Masked in logs
      run: |
        # Secret is masked: **** in logs
        echo "Connecting to database..."
        
    - name: Use with masking
      run: |
        echo "::add-mask::${{ secrets.API_KEY }}"  # Explicit masking
        ./deploy.sh --key ${{ secrets.API_KEY }}
```

**Kubernetes External Secrets** (Preview of Ch 49):
```yaml
apiVersion: external-secrets.io/v1beta1
kind: ExternalSecret
metadata:
  name: db-credentials
spec:
  refreshInterval: 1h
  secretStoreRef:
    kind: ClusterSecretStore
    name: vault-backend
  target:
    name: db-credentials
    creationPolicy: Owner
  data:
  - secretKey: password
    remoteRef:
      key: database/creds/app
      property: password
```

### Secret Scanning

Prevent secrets from entering repositories:

**Talisman** (Pre-commit hook):
```bash
# .talismanrc
fileignoreconfig:
- filename: config/secrets.yaml
  checksum: 8d2f3...
  ignore_detectors: [filecontent]
```

**TruffleHog** (CI pipeline):
```yaml
- name: Secret Detection
  uses: trufflesecurity/trufflehog@main
  with:
    path: ./
    base: main
    head: HEAD
    extra_args: --debug --only-verified
```

## 48.5 Compliance and Auditing

Continuous compliance ensures that every deployment meets regulatory requirements (SOC2, PCI-DSS, HIPAA, GDPR) without manual gatekeeping.

### Audit Logging

**Kubernetes Audit Policy**:
```yaml
apiVersion: audit.k8s.io/v1
kind: Policy
rules:
  # Log all requests to pods in production namespace
  - level: RequestResponse
    resources:
    - group: ""
      resources: ["pods"]
    namespaces: ["production"]
    
  # Log mutations to deployments
  - level: Metadata
    resources:
    - group: "apps"
      resources: ["deployments"]
    verbs: ["create", "update", "patch", "delete"]
    
  # Ignore system components
  - level: None
    users: ["system:kube-proxy"]
```

**CI/CD Audit Trail**:
```yaml
# GitHub Actions artifact retention
- name: Upload audit logs
  uses: actions/upload-artifact@v3
  with:
    name: pipeline-audit-${{ github.run_id }}
    path: |
      build.log
      test-results.xml
      sbom.json
      signed-attestation.json
    retention-days: 90  # Compliance requirement
```

### Compliance as Code

**Open Policy Agent (OPA)** for compliance checks:
```yaml
# Check for privileged containers (PCI-DSS requirement)
package kubernetes.admission

import rego.v1

deny contains msg if {
  input.request.kind.kind == "Pod"
  container := input.request.object.spec.containers[_]
  container.securityContext.privileged == true
  msg := sprintf("Container %s must not be privileged", [container.name])
}

deny contains msg if {
  input.request.kind.kind == "Pod"
  not input.request.object.spec.securityContext.runAsNonRoot
  msg := "Pod must run as non-root user"
}
```

**Compliance Scanning with kube-bench**:
```bash
# Run CIS Kubernetes Benchmark
kubectl apply -f https://raw.githubusercontent.com/aquasecurity/kube-bench/main/job.yaml

# Check results
kubectl logs job/kube-bench | grep -E "(FAIL|WARN)"
```

## 48.6 Security Policies

Policy-as-code automates security decisions, replacing manual reviews with automated enforcement.

### Pod Security Standards (PSS)

Kubernetes native replacement for PodSecurityPolicy:

**Restricted** (Most secure):
```yaml
apiVersion: v1
kind: Namespace
metadata:
  name: production
  labels:
    pod-security.kubernetes.io/enforce: restricted
    pod-security.kubernetes.io/audit: restricted
    pod-security.kubernetes.io/warn: restricted
```

**Baseline** (Minimal restrictions):
```yaml
labels:
  pod-security.kubernetes.io/enforce: baseline
```

**Privileged** (Unrestricted):
```yaml
labels:
  pod-security.kubernetes.io/enforce: privileged
```

### Falco (Runtime Security)

Detect anomalous behavior in running containers:

**Rule for Unexpected Connections**:
```yaml
- rule: Unexpected Outbound Connection
  desc: Detect outbound connections from production apps
  condition: >
    outbound and
    container and
    k8s.ns.name = "production" and
    not (k8s.label.app in (known_external_apps))
  output: >
    Unexpected connection from container
    (user=%user.name command=%proc.cmdline connection=%fd.name)
  priority: WARNING
```

**Integration with CI/CD**:
```bash
# Fail build if Falco detects violations during integration tests
falco -r rules.yaml -M 60 &  # Run for 60 seconds
sleep 5
./run-integration-tests.sh
if grep "Warning" /var/log/falco.log; then
  echo "Security violations detected"
  exit 1
fi
```

### Network Policies

Zero-trust networking between services:

```yaml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: api-allow-frontend-only
spec:
  podSelector:
    matchLabels:
      app: backend-api
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    - podSelector:
        matchLabels:
          app: frontend
    ports:
    - protocol: TCP
      port: 8080
  egress:
  - to:
    - podSelector:
        matchLabels:
          app: database
    ports:
    - protocol: TCP
      port: 5432
  - to:  # Allow DNS
    - namespaceSelector: {}
      podSelector:
        matchLabels:
          k8s-app: kube-dns
    ports:
    - protocol: UDP
      port: 53
```

## 48.7 Vulnerability Management

Continuous vulnerability scanning must happen at multiple stages: dependency resolution, image build, registry storage, and runtime execution.

### Scanning Stages

**1. Dependency Scanning (SCA)**:
```yaml
# OWASP Dependency-Check
- name: Dependency Check
  uses: dependency-check/Dependency-Check_Action@main
  with:
    project: 'my-app'
    path: '.'
    format: 'ALL'
    
# Snyk
- name: Snyk Test
  run: snyk test --severity-threshold=high
  env:
    SNYK_TOKEN: ${{ secrets.SNYK_TOKEN }}
```

**2. Container Image Scanning**:
```yaml
# Trivy
- name: Scan Image
  uses: aquasecurity/trivy-action@master
  with:
    image-ref: 'myapp:${{ github.sha }}'
    format: 'sarif'
    output: 'trivy-results.sarif'
    severity: 'CRITICAL,HIGH'
    exit-code: '1'  # Fail build on critical vulns
    
# Grype (Anchore)
- name: Grype Scan
  uses: anchore/scan-action@v3
  with:
    image: 'myapp:${{ github.sha }}'
    severity-cutoff: high
    fail-build: true
```

**3. Runtime Scanning**:
```bash
# Trivy Kubernetes operator scans running pods
kubectl apply -f https://raw.githubusercontent.com/aquasecurity/trivy-operator/main/deploy/static/trivy-operator.yaml

# View vulnerability reports
kubectl get vulnerabilityreports --all-namespaces
```

### Vulnerability Database Updates

Ensure scanners use latest vulnerability data:
```yaml
# Daily cache update job
apiVersion: batch/v1
kind: CronJob
metadata:
  name: vuln-db-update
spec:
  schedule: "0 2 * * *"
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: updater
            image: aquasec/trivy:latest
            command:
            - trivy
            - image
            - --download-db-only
          restartPolicy: OnFailure
```

### Remediation Workflows

**Automated Patching** (Dependabot/Renovate):
```yaml
# .github/dependabot.yml
version: 2
updates:
  - package-ecosystem: "docker"
    directory: "/"
    schedule:
      interval: "daily"
    open-pull-requests-limit: 10
    labels:
      - "security"
      - "dependencies"
      
  - package-ecosystem: "npm"
    directory: "/"
    schedule:
      interval: "weekly"
    allow:
      - dependency-type: "production"
    security-updates-only: true
```

## 48.8 Incident Response

When security incidents occur, CI/CD pipelines must support rapid investigation, containment, and recovery.

### Automated Response

**Image Rollback on CVE Detection**:
```yaml
# Policy: If runtime scan finds critical CVE, trigger rollback
apiVersion: automation.cloudcustodian.io/v1
kind: Policy
metadata:
  name: auto-rollback-critical-cve
spec:
  mode:
    type: cloudtrail
    events:
      - source: trivy-operator
        event: CriticalVulnerabilityFound
  actions:
    - type: rollback-deployment
      strategy: immediate
    - type: notify
      to: ["security@company.com"]
      subject: "Critical CVE detected in production - auto-rollback executed"
```

**Forensic Data Collection**:
```bash
# Capture container state before eviction
kubectl debug pod/compromised-pod --copy-to=forensic-pod --share-processes --image=busybox -- sleep 3600

# Export filesystem
kubectl cp forensic-pod:/tmp/forensics.tar.gz ./evidence.tar.gz

# Network capture
kubectl sniffer compromised-pod -p -f "port 443"
```

### Pipeline Segregation

**Break-Glass Procedures**:
```yaml
# Emergency deployment bypassing normal gates (requires MFA)
apiVersion: v1
kind: ConfigMap
metadata:
  name: break-glass-policy
data:
  policy: |
    if emergency_override and mfa_verified:
      allow_deployment(severity=critical)
      alert_security_team()
      require_post_incident_review(within=24h)
    else:
      enforce_standard_pipeline()
```

### Post-Incident Hardening

After incidents, update pipeline gates:
```yaml
# Add new policy based on incident learnings
- name: Prevent Reoccurrence
  uses: opa-policy-check@v1
  with:
    policy: |
      # Block specific vulnerable version
      deny if {
        input.image contains "log4j:2.14"
      }
```

---

## Chapter Summary and Preview

This chapter established comprehensive security practices for CI/CD pipelines, moving beyond perimeter defense to **Zero Trust** supply chain security. We examined **SBOMs** as essential inventory mechanisms for tracking software components, enabling rapid vulnerability response when new CVEs emerge. **Image signing** with Cosign and Sigstore provides cryptographic proof of provenance, while admission controllers enforce that only signed, scanned artifacts reach production. 

**Supply chain security** requires defending against dependency confusion, build system compromise, and poisoned pipelines through ephemeral environments, minimal privileges, and commit signing. **Vulnerability management** must be continuous—scanning dependencies during build, images in registry, and containers at runtime—rather than a one-time gate. **Policy-as-code** through OPA, Kyverno, and Pod Security Standards automates compliance, ensuring every deployment meets organizational security requirements without manual review bottlenecks.

**Key Takeaways:**
- Implement defense in depth: secure source code, build environment, artifacts, and runtime independently.
- Generate and store SBOMs for every build; they are essential for incident response and licensing compliance.
- Use keyless signing (Sigstore) rather than long-lived keys to reduce credential management overhead.
- Enforce image verification at the cluster level; never trust images based on tags alone.
- Shift security left with automated scanning in CI, but maintain runtime protection with Falco and network policies.
- Treat the CI/CD pipeline as a critical production system requiring the same security rigor as your applications.

**Security Culture:** Security is not a team or a phase—it is a property of the system. Build security controls that make the secure path the easiest path, automating compliance and verification so that developers can move fast without breaking things.

**Next Chapter Preview:** Chapter 49: Secrets Management dives deep into the secure handling of sensitive credentials within Kubernetes and CI/CD environments. We will explore Kubernetes Secrets limitations and the **External Secrets Operator** for cloud integration, HashiCorp **Vault** integration for dynamic secrets and database credential rotation, cloud-native secret managers (AWS Secrets Manager, Azure Key Vault, GCP Secret Manager), and GitOps-compatible secret patterns using **Sealed Secrets** and **Mozilla SOPS**. We will examine encryption at rest and in transit, secret rotation strategies without downtime, and avoiding common anti-patterns like environment variable injection and Git-committing encrypted secrets.